In [24]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("HUGGINGFACE_API_KEY")

llm = HuggingFaceEndpoint(repo_id="meta-llama/Llama-3.1-8B-Instruct", huggingfacehub_api_token=api_key)

model = ChatHuggingFace(llm=llm)

json_parser = JsonOutputParser()
str_parser = StrOutputParser()

# 1 Prompt - Generate detailed prompt on topic (Text Output)
template_01 = PromptTemplate(
    template="Write a detailed prompt on this {topic}.",
    input_variables=["topic"]
)


# 2nd Prompt - Summarize the result (JSON Output)
template_02 = PromptTemplate(
    template="Write a 5 lines summary on the following text.\n{text}\n{formate_intructions}",
    input_variables=["text"],
    partial_variables= {'formate_intructions': json_parser.get_format_instructions()}
)


# Single chain: topic -> detail (Text) -> summary (JSON)
chain = (
    template_01
    | model 
    | str_parser
    | (lambda x: {"text": x})
    | template_02 
    | model 
    | json_parser
)

result = chain.invoke("Black Holes")

print(result)



{'summary': ['Black holes are enigmatic and fascinating objects with an incredibly strong gravitational pull.', 'Their formation and evolution involve the collapse of massive stars and the role of dark matter.', 'Black holes have unique properties, including event horizons, ergospheres, and Hawking radiation.', 'Their effects on the surrounding environment include warping spacetime and bending light.', 'Detecting and observing black holes has significant implications for our understanding of gravity and the universe.']}
